# Explore Bootstrapping    

In [ ]:
# per condition
#     for surrogate count (1_000)
#         pick 100 trials with replacement
#         trimmed mean across those trials (20%)
#         left with one surrogate ERP
#     confidence interval at each timepoint, quantiles (0.25, 0.75)
#     # collapse across subjects
#     mean of all surrogates is the subject's ERP
# difference waves can be here
import mne
import glob
import numpy as np
import scipy.stats

In [ ]:
channel_interest = ['E75']
averaging_dict = {'VO21': [], 'VO23': []}
all_subject_paths = sorted(glob.glob('derivatives/segmented/*100*.fif'))
for sub in all_subject_paths:
    new_epoch = mne.read_epochs(sub).pick(channel_interest)
    for condition in averaging_dict.keys():
        averaging_dict[condition].append(new_epoch[condition])

In [ ]:
trim = lambda x: scipy.stats.trim_mean(x, 0.1, axis=0)
rng = np.random.default_rng(1)
bootstrapped_dict = {}
for condition in averaging_dict.keys():
    subject_accum = []
    for subject_epochs in averaging_dict[condition]:
        trial_cap = len(subject_epochs[condition])
        surrogate_accum = []
        for surrogate_count in range(100):
            surrogate = subject_epochs[rng.choice(trial_cap, trial_cap)].average(method=trim).data
            surrogate_accum.append(surrogate)
        surrogate_array = np.stack(surrogate_accum, axis=0)
        subject_accum.append(surrogate_array)
    subject_array = np.stack(subject_accum, axis=0)
    condition_array = np.mean(subject_array, axis=0)
    bootstrapped_dict[condition] = (np.percentile(condition_array, 25, axis=0),
                                    np.mean(condition_array, axis=0),
                                    np.percentile(condition_array, 75, axis=0))
print('Done')

In [ ]:
import plotly.express as px
import numpy as np
import plotly.graph_objects as go

times = averaging_dict['VO21'][0].times

df_builder = {'VO21_q25': bootstrapped_dict['VO21'][0][0],
              'VO21_mean': bootstrapped_dict['VO21'][1][0],
              'VO21_q75': bootstrapped_dict['VO21'][2][0],
              'VO23_q25': bootstrapped_dict['VO23'][0][0],
              'VO23_mean': bootstrapped_dict['VO23'][1][0],
              'VO23_q75': bootstrapped_dict['VO23'][2][0]}
df = pd.DataFrame(df_builder, index=times)

# Create the shaded region between y1 and y2
fig = go.Figure()

fig.add_trace(go.Scatter(x=df.index, y=df['VO21_q25'], mode='lines', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df.index, y=df['VO21_q75'], mode='lines', fill='tonexty', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df.index, y=df['VO21_mean'], mode='lines', line=dict(color='black')))

fig.add_trace(go.Scatter(x=df.index, y=df['VO23_q25'], mode='lines', line=dict(color='red')))
fig.add_trace(go.Scatter(x=df.index, y=df['VO23_q75'], mode='lines', fill='tonexty', line=dict(color='red')))
fig.add_trace(go.Scatter(x=df.index, y=df['VO23_mean'], mode='lines', line=dict(color='black')))

# Show the plot
fig.show()